<a href="https://colab.research.google.com/github/securitylab-repository/TPS-IA/blob/master/TP3_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP3

***L'objectif de ce TP est de vous montrer comment mener un projet de machine learning depuis la récupération du dataset jusqu'à l'étape de test en passant par la phase d'apprentissage.***

## Récupération du dataset
C'est une étape que nous avons eu l'occasion de voir lors des TPs précedents.

In [0]:
import pandas as pd
import numpy as np
from IPython.display import display

housing = pd.read_csv('https://raw.githubusercontent.com/securitylab-repository/TPS-IA/master/datasets/housing.csv',delimiter=',')

## Comprendre la structure du dataset

1. Affichage des cinq premiers exemples d'entrainement avec la fonction `head()` pour connaître les noms et le nombre d'attributs (features)

In [0]:
housing.head()

2. Il est également utile de connaître le nombre d'exemples d'entrainement, le nombre de valeurs non nulles et le type de chaque caractéristique (feature, colonne, attribut). A cet effet nous utilisons la méthode `info`

> Que remarquez-vous ?

In [0]:
housing.info()

***Réponse***

D'après le résultat de la fonction `info`, nous remarquons deux choses importantes:
- tous les features sont de type `float64` sauf l'attribut `ocean_proximity` qui est de type `object` (probablement un objet chaîne de caractères, car le `DataFrame` a été obtenu depuis un fichier CSV),

- il existe 207 exemples d'entrainement (lignes) qui ont une valeur nulle du feature `total_bedrooms`.

3. Sélectionnez un échantillon de quelques lignes et n'affichez que l'attribut `ocean_proximity'

> Que remarquez-vous ?

In [0]:
display(housing['ocean_proximity'].sample(100))

***Réponse***

On remarque que les valeurs de cet attribut se répétent, on est donc face à des catégories. 

4. Affichons le nombre d'occurences de chaque catégorie.

In [0]:
housing["ocean_proximity"].value_counts()

5. Affichage de la distribution (statistique) de tous les features numériques en utilisant la fonction `describe`

In [0]:
housing.describe()

Au delà des informations statistiques du dataset, la chose importante que nous pouvons noter ici, et qui influe considérablement sur la convergence des algorithmes d'apprentissage (notamment la décente en gradient), est que les features ont des échelles de valeurs différentes.  Ce qui nécessiterait par exemple une normalisation de tous les features (nous verrons cela plus loin).

6. Une autre façon d'analyser la distribution des valeurs de chaque feature est de dessiner l'hystogramme de chaque feature.

In [0]:
#%matplotlib inline   # Seulement dans Jupyter notebook
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

> Que remarquez-vous ?

***Réponse***

En plus des remarques soulevées précédement, nous constatons que les valeurs du feature `housing_median_age` et `media_house_value` sont plafonnées. Ceci peut causer de sérieux problèmes lors de l'apprentissage, surtout pour la colonne `media_house_value` qui est la cible à prédire. En effet, les algorithmes d'apprentissage risquent d'apprendre que les prix des maisons ne dépassent 500 000 euro.

La solutions est soit de revenir à l'état initial pour les exemples d'entrainement (lignes) concernés par ce plafonnement, soit de les supprimer de l'ensemble d'entrainement et de test.

![Texte alternatif…](https://github.com/securitylab-repository/TPS-IA/raw/master/histogramme_interpretation.png)


## Création de l'ensemble d'entrainement et de test

Lorsqu'on construit notre ensemble d'entrainement et de test, on fait attention au points suivants:

- En général on réserve 20% des données au jeux de test. Moins de 20% si le jeux de données est très large. Par conséquent 80% à l'ensemble d'entrainement.

- Le tirage des exemples doit être aléatoire et ne doit pas changer d'une exécution à une autre. Pour cela, le tirage pseudo aléatoire utilisé doit être paramétré avec une graine (seed) fixe.

- Le tirage doit prendre en considération les mises-à-jour du dataset et rester cohérent avec les tirages précédents (prendre seulement 20% parmi les nouveaux exemples d'entrainement).

Pour construire les ensembles d'entrainement et de test respectant ces contraintes, on utilisera la fonction `train_test_split` de la librairie `scikit-learn`.

> Que représente les arguments `test_size` et `random_state` ?

In [0]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
display(type(train_set))
display(type(test_set))
display(type(train_set.info()))
display(type(test_set.info()))

- Si le dataset n'est pas suffisament large, un simple tirage aléatoire ne suffit plus. Il faut prendre en cosidération le pourcentage de chaque catégorie influente de notre dataset de départ. Par exemple, si notre dataset sont les réponses d'un sondage sur une population constituée de 60% de fêmmes et 40% d'hommes, il est recommandé de tenir compte de ces pourcentage de départ dans la constitution de nos ensembles d'entrainement et de test. 




Supposons que selon un expert, l'attribut `media_income` est très déterminant dans la prédiction du prix d'une maison (l'attribut `median_house_value`). D'après l'histogramme de  `media_income`, nous pouvons remarquer que la majorité des exemples ont des valeurs comprises entre 1.5 et 6, mais il existe aussi des exemples dont les valeurs sont plus grande que 6. Il est donc intéressant  de piocher les exemples dans le dataset proportionnellement au nombre d'exemples dans chaque plage d'intervalle (catégorie à définir).
![Texte alternatif…](https://github.com/securitylab-repository/TPS-IA/raw/master/median_income.png)

A cet effet, nous utiliserons la fonction `pd.cut()`


In [0]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
housing["income_cat"].hist()

Nous allons maintenant reconstruire nos ensembles d'entrainement et de test en prenant en considération ces nouvelles catégories et en utilisant à présent la fonction `StratifiedShuffleSplit`.

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

strat_test_set["income_cat"].value_counts() / len(strat_test_set)

Supprimer maintenant la colonne que nous venons de créer pour rétablir l'état du dataset originel.

In [0]:
for set_ in (strat_train_set, strat_test_set): 
     set_.drop("income_cat", axis=1, inplace=True)

##  Analyse de la corrélation des données
### Coéfficient de corrélation standard (Pearson's R)

Dans le cas où le dataset n'est pas très large, il est intéressant de claculer la matrice de corrélation entre tous les attributs du dataset et en particulier entres les attributs et la cible.

Pour claculer la matrice de corrélaiton, on utilise la fonction `corr()` .

In [0]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)


> Interpréter les résultats ci-dessus ?

Une autre manière d'analyser les dépendances entre attributs est d'utiliser la fonction `scatter_matrix()`. Qui permet de visualiser graphiquement les attributs deux-à-deux. Le faire en prenant en considération tous les attributs, serait fastidieux. 

En général, on l'applique que sur les attributs les plus prometteurs (ceux qui auraient donnés de bons résultats avec la première méthode par exemple). 

In [0]:
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

> Ce résulat confirme-t-il celui obtenu avec la matrice de corrélation ?

On remarque que l'attribut qui montre le plus de corrélation avec notre cible `mediane_house_value` est `median_income`. Analysons de plus près le graphique associé.



In [0]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

> Quels problèmes remarquez-vous d'après ce résultat et comment y remédier ?

## Extraction des étiquettes (cibles)

Avant de continuer, il est utile de séparer les étiquettes du reste du `DataFrame`, car les transformations que nous allons effectuer sur l'un ou l'autre seront différentes.

In [0]:
housing = strat_train_set.drop("median_house_value", axis=1)
labels = strat_train_set["median_house_value"].copy()

## Nettoyage du dataset

Certains algorithmes de machine learning ne tolèrent pas de valeurs manquantes sur le dataset. Par exemple, dans le cas de ce dataset, l'attribut  `total_bedrooms` possèdent des valeurs manquantes.

Pour remédier à cela, nous avons trois solutions:

- Supprimer les lignes dont certaines valeurs sont manquantes
- Supprimer complètement l'attribut concerné 
- Mettre une autre valeur à la place du `null` (moyenne, médiane, zéro,...). Si on choisit cette dernière version, il faudrait sauvegarder la mediane calculée sur l'ensemble d'entrainement pour l'appliquer également sur l'ensemble de test. 




In [0]:
housing.isnull().sum()
#housing.dropna(subset=["total_bedrooms"])    # option 1
#housing.drop("total_bedrooms", axis=1)       # option 2

#median = housing["total_bedrooms"].median()  # option 3
#housing["total_bedrooms"].fillna(median, inplace=True)

Une autre façon de procéder est d'utiliser la classe `SimpleImputer`. Par contre elle fonctionne que sur des valeurs numériques. Il faut donc créer une copie des données sans l'attribut `ocean_proximity`.

In [0]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# faire un copie sans l'attribut ocean_proximity
housing_num = housing.drop("ocean_proximity", axis=1)

imputer.fit(housing_num)

print(imputer.statistics_)

print(housing_num.median().values)

X = imputer.transform(housing_num)

print(type(X))

housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
display(housing_tr.head())

> Vérifier qu'il n'existe plus de valeurs manquantes dans `housing_tr`

## Attribut sous forme de catégorie

Comme relevé ci-dessus, l'attribut `ocean_proximity` est de type `chaîne de caractères`, mais ses valeurs se répétent et ne sont pas nombreuses.

In [0]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

 On peut donc les regrouper par catégorie et représenter chaque catégorie par un entier. A cet effet, ont peut utiliser la classe `OrdinalEncoder` de `Scikit-Learn`.

In [0]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
print(housing_cat_encoded[:10])
print(ordinal_encoder.categories_)

Une autre manière de faire est de créer autant de colonnes que de catégories. Pour chaque ligne, la valeur de chaque colonne est 1 si la ligne appartient à cette catégorie ou 0 dans le cas contraire. 

In [0]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
print(housing_cat_1hot)
#print(housing_cat_1hot.toarray())
#print(cat_encoder.categories_)

> Comparez les deux méthodes

Une tout autre solution est de complètement modifier les valeurs de cette colonne en mettant à sa place des valeurs numériques proches de la signification de cette colonne. Dans cet exemple, on peut prendre par exemple la distance par rapport à l'océan. Ceci donnerait un sens proche de celui que représentent les catégories. 

## Transformation des features

Il arrive assez souvent qu'on veuille transformer ou même ajouter de nouveaux features plus pertinents à notre dataset.

Pour ce faire et afin d'automatiser cette tâche, Nous allons créer une classe permettant de le faire. 

Cette classe va hériter des classes  `BaseEstimator`, `TransformerMixin` afin de mieux l'intégrer à scikit-learn



In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [0]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

> Expliquez ce que permet de réaliser cette classe

## Feature Scaling

Comme indiqué dans le cours, il est important pour certains algorithmes d'apprentissage automatique d'opérer sur des données dont les valeurs sont à-peu près dans la même echelle de grandeur. Il existe deux méthodes pour remédier à ce problème :
- La standardisation (On peut utiliser la classe `StandardScaler`)
- La normalisation:  


In [0]:
housing_mean = housing.drop("ocean_proximity", axis=1)
for col in housing.columns:
  if (col != "ocean_proximity"):
    housing_mean[col] = (housing_mean[col] - housing_mean[col].median()) / (housing_mean[col].max() - housing_mean[col].min())

print(housing_mean.head())

> Donnez la différence entre les deux méthodes en demandant à google

## Transformation en Pipelines

Vous avez remarqué qu'il y a de nombreuses transformations que nous devons réaliser dans un ordre bien précis. Heureusement que Scikit-learn offre un moyen automatisé pour le faire et dans le bon ordre. Il le fait à travers la classe `Pipeline`.

Exemple: Effectuons les transformations suivantes sur les features numériques en nous aidant d'un `Pipeline`:
- Supprimer les lignes dont certaines valeurs sont `NaN`
- Combiner certains attributs 
- Standardisation

Le constructeur de la classe `Pipeline` admet une liste d'objets appelés `estimateur/transformer`. La particularité de ces objets est qu'ils doivent posséder une fonction `fit_transform()` et `fit()`. 

La classe `PipeLine` appelle  dansl l'ordre la fonction `fit_transform()` de chaque estimateur de la liste en lui transmettant à chaque fois le résultat retourné par l'estimateur précédent.

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num = housing.drop("ocean_proximity", axis=1)
housing_num_tr = num_pipeline.fit_transform(housing_num)
print(housing_num_tr)

> D'où vient la classe `CombinedAttributesAdder` ?

Et si on veut traiter au même temps les features numériques et non numériques, on peut utiliser la classe `ColumnTransformer`. Contrairement à la classe `Pipeline`, son constructeur admet en plus la liste des features concernés par la transformation.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)
print(housing_prepared)

## Entraînement et évaluation sur l'ensemble d'entraînement

Nous allons utiliser ici la régression linéaire.

In [0]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
some_data = housing.iloc[:5]
some_labels = labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))

Predictions: [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]


Essayons maintenant sur l'ensemble du dataset

In [0]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

> Que pensez vous du résultat obtenu, sommes-nous face à un overfiting ou underfiting


Essayon un autre algorithme qui donne de meilleurs résultats que ce soit sur des données linéairement séparables ou non, à savoir les arbres de décision.

In [0]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Evaluons maintenant l'algorithme sur les données d'entraînement

In [0]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

Que pensez vous du résultat obtenu, sommes-nous face à un overfiting ou à un underfiting ?

## Cross validation

Il arrive souvent que le choix de l'algorithme (Estimateur) à utiliser n'est pas évident. Pour choisir la meilleure solution à entraîner avant la phase de test, on utilise ce qu'on appelle la validation croisée (ex. `K-fold cross-validation`). On subdivise le dataset d'entrainement en $n$ partitions (appelées `folds`) et on réalise $k$ itérations entrainement/validation. A chaque itération, on choisit un `fold` différent pour les tests et les `k-1` restant pour l'entrainement.

### Validation croisée (Régression linéaire)

In [0]:
def display_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Standard deviation:", scores.std())

In [0]:
from sklearn.model_selection import cross_val_score
lin_scores = cross_val_score(lin_reg, housing_prepared, labels,
scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

### Validation croisée (Arbre de décision)

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

display_scores(tree_rmse_scores)

> Que signifie la ligne `Standrd deviation` ? 

> Pourquoi avons-nous de meilleurs résultats avec la régression linéaire ?

### Validation croisée (Random Forest) 

L'algorithme `Random Forest` est réputé pour donner de meilleurs résultats que les simples arbres de décision. Il construit plusieurs arbres de décision sur un sous ensembles de features à chaque fois et réalise une moyenne.

> Réalisez l'apprentissage d'un algorithme `Random Forest` (utilisez la classe `RandomForestRegressor`) 

> Faites une prédiction sur l'ensemble d'entrainement. Sommes-nous face à un overfiting ou à un underfiting ?

> Faites une validation croisée 


In [0]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, labels)

In [0]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [0]:
scores = cross_val_score(forest_reg, housing_prepared, labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

display_scores(forest_rmse_scores)

##  Grid Search

Une fois un ou plusieurs algorithmes sélectionnés, nous aurons souvent, selon les algorithmes, certains hyper-paramètres à positionner. Par exemple, nous utiliserons ici la validation croisée pour choisir les meilleures valeurs des hyper-paramètres `n_estimators` et `max_features` de l'algorithme `Random Forest`. 

In [0]:
from sklearn.model_selection import GridSearchCV

# Dictionnaire définissant les paramètres à évaluer
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, labels)

Affichage des meilleurs valeurs des hyper-paramètres trouvés

In [0]:
grid_search.best_params_


Sauvegrde de la meilleure configuration de l'algorithme 

In [0]:
model_final = grid_search.best_estimator_

## Sélection des meilleurs features

Une autre façon d'utiliser la validation croisée est de sélectionner les features les plus influents dans l'apprentissage afin de réduire la dimensionnalité du dataset. Ceci est souvent nécessaire quand le nombre de features est très important. Ce n'est pas vraiment le cas ici. 

In [0]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [0]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

## Evaluation sur l'ensemble de test

Il est maintenant temps d'évaluer l'agorithme retenu, à savoir ici l'algorithme `Random Forest`, sur l'ensemble de test.

In [0]:
# Le meilleur paramétrage de l'algorithme Random Forest défini précédement 
model_final = grid_search.best_estimator_

# Ensemble de test
X_test = strat_test_set.drop("median_house_value", axis=1)

# Les étiquettes 
y_test = strat_test_set["median_house_value"].copy()

# Application des transformations retenues plus en haut sur l'ensemble de test, en appliquant le Pipeline défini précédement
X_test_prepared = full_pipeline.transform(X_test)

# Validation
final_predictions = model_final.predict(X_test_prepared)

# Calcul des erreurs de prédiction
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)   
print(final_rmse)

# Exercice

Appliquez la même démarche que précédement sur ce [dataset](https://github.com/securitylab-repository/TPS-IA/raw/master/payment_fraud.csv).

Vous devez choisir entre les algorithmes:

  - Régression logistique
  - Les $K$ Plus proches voisins ($Knn$)